# Lesson 1: Advanced RAG Pipeline

In [1]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [2]:
from llama_index import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [3]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

41 

<class 'llama_index.schema.Document'>
Doc ID: c396f0b9-d23f-4100-b8fb-03f2857bf0e4
Text: PAGE 1Founder, DeepLearning.AICollected Insights from Andrew Ng
How to  Build Your Career in AIA Simple Guide


## Basic RAG pipeline

In [4]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [5]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm, embed_model="local:BAAI/bge-small-en-v1.5"
)
index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
query_engine = index.as_query_engine()

In [7]:
response = query_engine.query(
    "What are steps to take when finding projects to build your experience?"
)
print(str(response))

Develop a side hustle, ensure the project will help you grow technically, collaborate with good teammates, and consider if the project can be a stepping stone to larger projects.


## Evaluation setup using TruLens

In [8]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        print(item)
        eval_questions.append(item)

What are the keys to building a career in AI?
How can teamwork contribute to success in AI?
What is the importance of networking in AI?
What are some good habits to develop for a successful career?
How can altruism be beneficial in building a career?
What is imposter syndrome and how does it relate to AI?
Who are some accomplished individuals who have experienced imposter syndrome?
What is the first step to becoming good at AI?
What are some common challenges in AI?
Is it normal to find parts of AI challenging?


In [9]:
# You can try your own question:
new_question = "What is the right AI job for me?"
eval_questions.append(new_question)

In [10]:
print(eval_questions)

['What are the keys to building a career in AI?', 'How can teamwork contribute to success in AI?', 'What is the importance of networking in AI?', 'What are some good habits to develop for a successful career?', 'How can altruism be beneficial in building a career?', 'What is imposter syndrome and how does it relate to AI?', 'Who are some accomplished individuals who have experienced imposter syndrome?', 'What is the first step to becoming good at AI?', 'What are some common challenges in AI?', 'Is it normal to find parts of AI challenging?', 'What is the right AI job for me?']


In [11]:
from trulens_eval import Tru
tru = Tru()

tru.reset_database()

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `Tru` to prevent this.


For the classroom, we've written some of the code in helper functions inside a utils.py file.  
- You can view the utils.py file in the file directory by clicking on the "Jupyter" logo at the top of the notebook.
- In later lessons, you'll get to work directly with the code that's currently wrapped inside these helper functions, to give you more options to customize your RAG pipeline.

In [12]:
from utils import get_prebuilt_trulens_recorder

tru_recorder = get_prebuilt_trulens_recorder(query_engine,
                                             app_id="Direct Query Engine")

In [13]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [14]:
records, feedback = tru.get_records_and_feedback(app_ids=[])

In [15]:
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Context Relevance,Groundedness,Answer Relevance_calls,Context Relevance_calls,Groundedness_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_181eb45801ecf39b5b2e4727d4a085c3,"""What are the keys to building a career in AI?""","""Learning foundational technical skills, worki...",-,"{""record_id"": ""record_hash_181eb45801ecf39b5b2...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-02-08T14:51:03.514169"", ""...",2025-02-08T14:51:04.948049,1.0,1.0,1.000000,[{'args': {'prompt': 'What are the keys to bui...,[{'args': {'prompt': 'What are the keys to bui...,"[{'args': {'source': 'PAGE 1Founder, DeepLearn...",1,2109,0.003210
1,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_800ab39cb9a675601e85a4be114f4887,"""How can teamwork contribute to success in AI?""","""Teamwork can contribute to success in AI by a...",-,"{""record_id"": ""record_hash_800ab39cb9a675601e8...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-02-08T14:51:05.074438"", ""...",2025-02-08T14:51:06.166661,1.0,0.5,0.333333,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'prompt': 'How can teamwork contrib...,[{'args': {'source': 'Hopefully the previous c...,1,1678,0.002544
2,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_a06e3e5b57ef836f84c40b99aea56c73,"""What is the importance of networking in AI?""","""Networking is crucial in AI as it helps indiv...",-,"{""record_id"": ""record_hash_a06e3e5b57ef836f84c...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-02-08T14:51:06.294298"", ""...",2025-02-08T14:51:07.427404,1.0,0.5,0.666667,[{'args': {'prompt': 'What is the importance o...,[{'args': {'prompt': 'What is the importance o...,[{'args': {'source': 'Hopefully the previous c...,1,1696,0.002580
3,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_1a9cd64a81c2a9c6b23fca5ef61ffaaa,"""What are some good habits to develop for a su...","""Developing good habits in areas such as eatin...",-,"{""record_id"": ""record_hash_1a9cd64a81c2a9c6b23...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-02-08T14:51:07.569626"", ""...",2025-02-08T14:51:08.436103,1.0,1.0,NaN,[{'args': {'prompt': 'What are some good habit...,[{'args': {'prompt': 'What are some good habit...,NaN,0,1632,0.002466
4,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_c774e15fb160c582f7fcbfe5dfef9632,"""How can altruism be beneficial in building a ...","""Helping others during your career journey can...",-,"{""record_id"": ""record_hash_c774e15fb160c582f7f...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2025-02-08T14:51:08.561385"", ""...",2025-02-08T14:51:09.212956,1.0,NaN,NaN,[{'args': {'prompt': 'How can altruism be bene...,NaN,NaN,0,1610,0.002423


In [16]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at https://s172-29-54-254p38560.lab-aws-production.deeplearning.ai/ .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Advanced RAG pipeline

### 1. Sentence Window retrieval

In [17]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

In [18]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

In [19]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [20]:
window_response = sentence_window_engine.query(
    "how do I get started on a personal project in AI?"
)
print(str(window_response))

You can begin a personal project in AI by following the steps outlined in the chapters provided. Start by identifying and scoping AI projects that align with your career goals. Ensure that the projects you choose are responsible, ethical, and beneficial to people. As you progress, aim to work on projects that increase in scope, complexity, and impact over time. Building a portfolio of projects that demonstrate skill progression is also essential. Finally, consider using a simple framework for starting your AI job search to further develop your expertise in the field.


In [21]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine,
    app_id = "Sentence Window Query Engine"
)

In [22]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        print(question)
        print(str(response))

What are the keys to building a career in AI?
Learning foundational technical skills, working on projects, finding a job, and being part of a supportive community are the keys to building a career in AI.
How can teamwork contribute to success in AI?
Teammates play a crucial role in the success of AI projects. Working collaboratively with colleagues who are dedicated, continuously learning, and focused on building AI for the benefit of all can positively influence individual performance. The ability to work effectively within a team, leveraging each member's strengths and insights, can lead to improved project outcomes and overall success in the field of AI.
What is the importance of networking in AI?
Networking in AI is crucial as it can provide valuable insights, guidance, and opportunities for individuals looking to advance in the field. By connecting with professionals who have experience in AI, individuals can gain knowledge about the industry, potential career paths, and current t

In [23]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
Sentence Window Query Engine,0.36875,1.0,0.48,1.272727,0.000818


In [24]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-29-54-254p38560.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

### 2. Auto-merging retrieval

In [25]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index"
)

In [26]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [27]:
auto_merging_response = automerging_query_engine.query(
    "How do I build a portfolio of AI projects?"
)
print(str(auto_merging_response))

> Merging 1 nodes into parent node.
> Parent node id: d20193f4-d296-44f1-81d8-daf4f606ded7.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

> Merging 1 nodes into parent node.
> Parent node id: cea27ff2-a7e4-4ac4-b6f4-470f7a427f8c.
> Parent node text: PAGE 21Building a Portfolio of 
Projects that Shows 
Skill Progression CHAPTER 6
PROJECTS

Building a portfolio of AI projects involves showcasing a progression from simple to complex undertakings over time. It is important to be able to communicate your thinking effectively to demonstrate the value of your work and gain the trust of others. Identifying ideas that are worth working on is a crucial skill for an AI architect, and working on projects across various industries can help in gaining experience and building a diverse portfolio.


In [28]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(automerging_query_engine,
                                                         app_id="Automerging Query Engine")

In [29]:
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        print(question)
        print(response)

A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f916552d450 is calling an instrumented method <function BaseQueryEngine.query at 0x7f92d37a7640>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f92501963b0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f916552d450 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f92ce60aa70>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f92501963b0) using this function.
A new object of type <class 'llama_index.retrievers.auto_merging_retriever.AutoMergingRetriever'> at 0x7f91c83e4af0 is calling an instrumented method <function BaseRetriever.retrieve at 0x7f92d37a69e0>. The path of this call may be incorrect.
Guessing path of new object is app.retriever based on other object (0x7

> Merging 2 nodes into parent node.
> Parent node id: 817dd243-2f67-4bb8-8562-c01ae3712aa3.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...

> Merging 1 nodes into parent node.
> Parent node id: e780d561-5342-4a28-938d-4ec80a0529f0.
> Parent node text: PAGE 3Table of 
ContentsIntroduction: Coding AI is the New Literacy.
Chapter 1: Three Steps to Ca...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f916552d3c0 is calling an instrumented method <function CompactAndRefine.get_response at 0x7f92d2dc3520>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f92501978e0) using this function.
A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f916552d3c0 is calling an instrumented method <function Refine.get_response at 0x7f92d1fdf2e0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f92501978e0) using this function.
A new object of type <class 'llama_index.llm_predictor.base.LLMPredictor'> at 0x7f92784b2ec0 is calling an instrumented method <function LLMPredictor.predict at 0x7f92dfca1120>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthes

What are the keys to building a career in AI?
Learning foundational technical skills, working on projects, finding a job, and being part of a community are key steps to building a career in AI. Additionally, collaborating with others, influencing, and being influenced by others are crucial aspects for success in AI.


A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f916552d3c0 is calling an instrumented method <function Refine.get_response at 0x7f92d1fdf2e0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f92501978e0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f916552d450 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f92ce60aa70>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f92501963b0) using this function.


...

What are some common challenges in AI?
Some common challenges in AI include the highly iterative nature of AI projects, which can make it difficult to plan accurately for building a system, and technical challenges that many individuals face when working on AI projects.
> Merging 3 nodes into parent node.
> Parent node id: 8e4b6d39-00a1-4ed4-b738-6f1fc8f1f0d5.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...

> Merging 1 nodes into parent node.
> Parent node id: 70a8c9b1-8690-4280-b60a-4258a0eefe16.
> Parent node text: PAGE 38Before we dive into the final chapter of this book, I’d like to address the serious matter...



A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f916552d3c0 is calling an instrumented method <function Refine.get_response at 0x7f92d1fdf2e0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f92501978e0) using this function.
A new object of type <class 'llama_index.query_engine.retriever_query_engine.RetrieverQueryEngine'> at 0x7f916552d450 is calling an instrumented method <function RetrieverQueryEngine.retrieve at 0x7f92ce60aa70>. The path of this call may be incorrect.
Guessing path of new object is app based on other object (0x7f92501963b0) using this function.


Is it normal to find parts of AI challenging?
It is normal to find parts of AI challenging.
> Merging 1 nodes into parent node.
> Parent node id: e32014c9-4d85-44dd-a97a-4974a18557d0.
> Parent node text: PAGE 31Finding the Right 
AI Job for YouCHAPTER 9
JOBS

> Merging 1 nodes into parent node.
> Parent node id: 9991a193-58db-42ac-b0af-72aeb2edb46f.
> Parent node text: If you’re leaving 
a job, exit gracefully. Give your employer ample notice, give your full effort...

> Merging 1 nodes into parent node.
> Parent node id: 5479ec3a-ea32-4892-9a7a-dbf28081fefb.
> Parent node text: PAGE 28Using Informational 
Interviews to Find 
the Right JobCHAPTER 8
JOBS

> Merging 1 nodes into parent node.
> Parent node id: 6464ab14-f1fd-41e9-971f-68947a3f2fb2.
> Parent node text: PAGE 31Finding the Right 
AI Job for YouCHAPTER 9
JOBS

> Merging 1 nodes into parent node.
> Parent node id: a271d08c-64be-43ff-986e-cdcb5f212ee5.
> Parent node text: PAGE 28Using Informational 
Interviews to Find 
the Right

A new object of type <class 'llama_index.response_synthesizers.compact_and_refine.CompactAndRefine'> at 0x7f916552d3c0 is calling an instrumented method <function Refine.get_response at 0x7f92d1fdf2e0>. The path of this call may be incorrect.
Guessing path of new object is app._response_synthesizer based on other object (0x7f92501978e0) using this function.


What is the right AI job for me?
The right AI job for you would likely depend on whether you are looking to switch roles, industries, or both. If you are seeking your first job in AI, transitioning either roles or industries individually may be easier than attempting both simultaneously.


In [30]:
tru.get_leaderboard(app_ids=[])

,Groundedness,Answer Relevance,Context Relevance,latency,total_cost
app_id,,,,,
Automerging Query Engine,0.795,0.981818,0.69375,2.181818,0.000862


In [31]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path: https://s172-29-54-254p38560.lab-aws-production.deeplearning.ai/


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>